# Código do dashboard
## Autores: Agnaldo B. Júnior e Pedro H. C. dos Santos

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import psycopg2
necessaryKeys = ["id","ASIN","title","group","salesrank","similar","categories","reviews"]

con = psycopg2.connect( user = "postgres",
                        password = "postgres",
						host = "localhost",
						port = "5432",
						dbname = "amazon")
cursor = con.cursor()
# cursor.execute('DELETE FROM reviews;DELETE FROM product_category;DELETE FROM categories;DELETE FROM similars;DELETE FROM products;')
con.commit()


cursor.close()
con.close()

In [18]:
#Questão 5a:
def getHelpfulReviews(asin):
    con = psycopg2.connect( user = "postgres",
                            password = "postgres",
                            host = "localhost",
                            port = "5432",
                            dbname = "amazon")
    cursor = con.cursor()
    cursor.execute('''select * from (
                  (select * from products join reviews r on products.asin = r.product_asin where asin='{}' order by helpful DESC , rating DESC LIMIT 5)
                  union all
                  (select * from products join reviews r on products.asin = r.product_asin where asin='{}' order by rating DESC ,helpful DESC  LIMIT 5)
              ) result;'''.format(asin,asin))
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    return rows
getHelpfulReviews('0738700797')

[(2,
  '0738700797',
  'Candlemas: Feast of Flames',
  'Book',
  168596,
  338314,
  datetime.date(2002, 2, 6),
  'A2P6KAWXJ16234',
  4,
  16,
  16,
  '0738700797'),
 (2,
  '0738700797',
  'Candlemas: Feast of Flames',
  'Book',
  168596,
  338320,
  datetime.date(2004, 2, 11),
  'A1CP26N8RHYVVO',
  1,
  13,
  9,
  '0738700797'),
 (2,
  '0738700797',
  'Candlemas: Feast of Flames',
  'Book',
  168596,
  338312,
  datetime.date(2002, 1, 24),
  'A13SG9ACZ9O5IM',
  5,
  8,
  8,
  '0738700797'),
 (2,
  '0738700797',
  'Candlemas: Feast of Flames',
  'Book',
  168596,
  338317,
  datetime.date(2002, 5, 23),
  'A1GIL64QK68WKL',
  5,
  8,
  8,
  '0738700797'),
 (2,
  '0738700797',
  'Candlemas: Feast of Flames',
  'Book',
  168596,
  338316,
  datetime.date(2002, 3, 23),
  'A3GO7UV9XX14D8',
  4,
  6,
  6,
  '0738700797'),
 (2,
  '0738700797',
  'Candlemas: Feast of Flames',
  'Book',
  168596,
  338317,
  datetime.date(2002, 5, 23),
  'A1GIL64QK68WKL',
  5,
  8,
  8,
  '0738700797'),
 (2,
  '

In [17]:
#Questao 5b:
def getSimilarsWithMostSalesrank(asin):
    con = psycopg2.connect( user = "postgres",
                            password = "postgres",
                            host = "localhost",
                            port = "5432",
                            dbname = "amazon")
    cursor = con.cursor()
    cursor.execute('''select * from (similars JOIN products ON similars.asin_1 = products.asin) AS result
    JOIN products ON result.asin_2 = products.asin  where asin_1='{}' and result.salesrank < products.salesrank;'''.format(asin))
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    return rows
getSimilarsWithMostSalesrank('0738700797')

[('0738700797',
  '0738700827',
  2,
  '0738700797',
  'Candlemas: Feast of Flames',
  'Book',
  168596,
  454888,
  '0738700827',
  'Ostara: Customs, Spells & Rituals for the Rites of Spring',
  'Book',
  311254),
 ('0738700797',
  '1567182836',
  2,
  '0738700797',
  'Candlemas: Feast of Flames',
  'Book',
  168596,
  411179,
  '1567182836',
  'Beltane: Springtime Rituals, Lore and Celebration',
  'Book',
  280541)]

In [48]:
#Questao 5c:
def getProductAvgTimeline(asin):
    con = psycopg2.connect( user = "postgres",
                            password = "postgres",
                            host = "localhost",
                            port = "5432",
                            dbname = "amazon")
    cursor = con.cursor()
    cursor.execute('''select date,rating from products join reviews r on products.asin = r.product_asin where asin='{}' order by date;'''.format(asin))
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    ratings = []
    for index in range(0,len(rows)):
        avg = 0
        for j in range(0,index+1):
            avg += rows[j][1]
        avg = avg/(index+1)
        ratings.append((rows[index][0],avg))
    return ratings
getProductAvgTimeline('0738700797')

[(datetime.date(2001, 12, 16), 5.0),
 (datetime.date(2002, 1, 7), 4.5),
 (datetime.date(2002, 1, 24), 4.666666666666667),
 (datetime.date(2002, 1, 28), 4.75),
 (datetime.date(2002, 2, 6), 4.6),
 (datetime.date(2002, 2, 14), 4.5),
 (datetime.date(2002, 3, 23), 4.428571428571429),
 (datetime.date(2002, 5, 23), 4.5),
 (datetime.date(2003, 2, 25), 4.555555555555555),
 (datetime.date(2003, 11, 25), 4.6),
 (datetime.date(2004, 2, 11), 4.2727272727272725),
 (datetime.date(2005, 2, 7), 4.333333333333333)]

In [53]:
#Questao 5d:
def getTop10ProductsOfEachGroup():
    con = psycopg2.connect( user = "postgres",
                            password = "postgres",
                            host = "localhost",
                            port = "5432",
                            dbname = "amazon")
    cursor = con.cursor()
    cursor.execute('''SELECT * FROM (
                SELECT *, rank() OVER (
                    PARTITION BY products."group"
                    ORDER BY products.salesrank DESC
                )
                FROM products
                WHERE salesrank notnull
                ) jobs WHERE  RANK <=10;''')
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    return rows
getTop10ProductsOfEachGroup()

[(197564,
  '0880881658',
  "Baby'S Record Keeper And Memory Box",
  'Baby Product',
  1017,
  1),
 (392331, '0525445404', 'Planes (A Unicorn Paperback)', 'Book', 3798351, 1),
 (293111,
  '9994537873',
  'The Efl/Esl Job Search Handbook: Included Is a Step-By-Step Approach to the Efl/Esl Job Search',
  'Book',
  3796990,
  2),
 (408579,
  '0821812718',
  'Algebraic Groups and Modular Lie Algebras',
  'Book',
  3795269,
  3),
 (4568,
  '0413501000',
  'Favourite Nights and Caught on a Train (Methuen New Theatrescript)',
  'Book',
  3793303,
  4),
 (242292,
  '3540971076',
  'How to Measure Angles from Foot Radiographs',
  'Book',
  3788732,
  5),
 (45743, '0335002633', 'Organizations As Systems', 'Book', 3781483, 6),
 (396871,
  '0306404796',
  'Recent Developments in Gauge Theories (Nato Science Series: B:)',
  'Book',
  3779983,
  7),
 (450097, '0685500942', 'Fish and Fisheries of India', 'Book', 3779920, 8),
 (446568, '0917062043', 'Arabic: Phonology and Script', 'Book', 3773956, 9),

In [54]:
#Questao 5e:
def getTop10ProductsWithMostRatingAVG():
    con = psycopg2.connect( user = "postgres",
                            password = "postgres",
                            host = "localhost",
                            port = "5432",
                            dbname = "amazon")
    cursor = con.cursor()
    cursor.execute('''SELECT asin,title,avg(helpful) FROM products join reviews r on products.asin = r.product_asin 
    GROUP BY (asin,title) order by avg(helpful) DESC LIMIT 10;''')
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    return rows
getTop10ProductsWithMostRatingAVG()


[('0967699703',
  'Understanding Loved Boys and Boylovers',
  Decimal('243.0000000000000000')),
 ('188553891X', "T'ai Chi for Older Adults", Decimal('233.0000000000000000')),
 ('0960316833',
  'More Than Just Hot Air: Common Sense Counter-Top Convection Oven Cookbook',
  Decimal('203.0000000000000000')),
 ('1575025809',
  'Crockpot Cookery (Cookbooks By Morris Press)',
  Decimal('197.0000000000000000')),
 ('0968519903',
  'Creative Interventions for Troubled Children & Youth',
  Decimal('196.0000000000000000')),
 ('0140502416', 'The Story About Ping', Decimal('186.4651162790697674')),
 ('0448421658',
  'The Story About Ping (8x 8)',
  Decimal('186.3488372093023256')),
 ('0140951172',
  'The Story about Ping : StoryTape (StoryTape, Puffin)',
  Decimal('186.3255813953488372')),
 ('0670672238', 'The Story about Ping', Decimal('186.3023255813953488')),
 ('0811701166',
  'The Smoked-Foods Cookbook: How to Flavor, Cure, and Prepare Savory Meats, Game, Fish, Nuts, and Cheese',
  Decimal('183.

In [56]:
#Questao 5f:
def getTop5CategoriesWithMostRating():
    con = psycopg2.connect( user = "postgres",
                            password = "postgres",
                            host = "localhost",
                            port = "5432",
                            dbname = "amazon")
    cursor = con.cursor()
    cursor.execute('''select category_id,name,avg
        from (
         SELECT category_id, avg(helpful)
         FROM products
                  join reviews r on products.asin = r.product_asin
                  join product_category pc on products.asin = pc.product_asin
                  join categories c on pc.category_id = c.id
         group by (category_id)
         order by avg(helpful) desc
         limit 5
        ) result join categories on category_id=id;''')
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    return rows
getTop5CategoriesWithMostRating()

[(727732, 'CasualUsers', Decimal('84.5000000000000000')),
 (502030, 'CasualUsers', Decimal('84.5000000000000000')),
 (4816, 'DominicanRepublic', Decimal('78.6949152542372881')),
 (20197, 'Goetz,Hermann', Decimal('75.0000000000000000')),
 (502256, 'Advanced', Decimal('68.0000000000000000'))]

In [59]:
#Questao 5g:
def get10CustomersWithMostReviewsForEachGroup():
    con = psycopg2.connect( user = "postgres",
                            password = "postgres",
                            host = "localhost",
                            port = "5432",
                            dbname = "amazon")
    cursor = con.cursor()
    cursor.execute('''SELECT customer,"group",count FROM (
                SELECT *, row_number()  OVER (
                    PARTITION BY "group"
                    ORDER BY count DESC
                )
                FROM ( SELECT count(customer),customer,"group" FROM reviews,products WHERE product_asin = asin GROUP BY customer,"group" ORDER BY "group",count(customer) DESC) xuei
            ) jobs WHERE row_number <= 10;''')
    rows = cursor.fetchall()
    cursor.close()
    con.close()
    return rows
get10CustomersWithMostReviewsForEachGroup()


[('AI9SB5VKUFXDC', 'Baby Product', 1),
 ('A37TFIP0OMKGMW', 'Baby Product', 1),
 ('A2LAH8VX720175', 'Baby Product', 1),
 ('ATVPDKIKX0DER', 'Book', 643185),
 ('A3UN6WX5RRO2AG', 'Book', 154531),
 ('A14OJS0VWMOSWO', 'Book', 9589),
 ('AFVQZQ8PW0L', 'Book', 5441),
 ('A1K1JW1C5CUSUZ', 'Book', 3562),
 ('A2NJO6YE954DBH', 'Book', 2055),
 ('A3QVAKVRAH657N', 'Book', 1651),
 ('A1NATT3PN24QWY', 'Book', 1535),
 ('A1D2C0WDCSHUWZ', 'Book', 1508),
 ('A2ODBHT4URXVXQ', 'Book', 1469),
 ('A1SFX3CR838F36', 'CE', 1),
 ('A2IX9TMXDBUCYV', 'CE', 1),
 ('A13JU90C7AU3RT', 'CE', 1),
 ('A1328SYT22GA4U', 'CE', 1),
 ('A1J62O1S6QTHZJ', 'CE', 1),
 ('ATVPDKIKX0DER', 'DVD', 63148),
 ('A3UN6WX5RRO2AG', 'DVD', 15549),
 ('A2NJO6YE954DBH', 'DVD', 1366),
 ('AU8552YCOO5QX', 'DVD', 1213),
 ('A3P1A63Q8L32C5', 'DVD', 859),
 ('A3LZGLA88K0LA0', 'DVD', 856),
 ('A82LIVYSX6WZ9', 'DVD', 683),
 ('A152C8GYY25HAH', 'DVD', 675),
 ('A16CZRQL23NOIW', 'DVD', 651),
 ('A1CZICCYP2M5PX', 'DVD', 650),
 ('ATVPDKIKX0DER', 'Music', 166149),
 ('A3UN6WX5